# Mesh to Mesh Registration Example

ITK natively supports image-to-image registration, which is a common operation for medical images with symmetry. Another common method of storing 3D volumetric data is to represent volume surfaces with meshes. In this example we seek to register two meshes using various metrics and optimization techniques built on top of ITK.

Registration classes are defined in the Python `hasi` submodule and built on top of the ITK Python wrapping. The `MeanSquaresRegistrar`, `DiffeoRegistrar`, and `ElastixRegistrar` classes apply registration techniques to images derived from mesh inputs, while the `PointSetEntropyRegistrar` aims to register meshes via point set metrics. Mesh registration is carried out with each class in this notebook on sample bone femur mesh data in the `examples/Data` folder.

This notebook requires the following modules, which can be either acquired via `pip` or built alongside the ITK `master` branch:
- [ITK](https://github.com/InsightSoftwareConsortium/ITK/)
- [ITKBoneEnhancement](https://github.com/InsightSoftwareConsortium/ITKBoneEnhancement)
- [ITKMeshToPolyData](https://github.com/InsightSoftwareConsortium/ITKMeshToPolyData)
- [ITKElastix](https://github.com/InsightSoftwareConsortium/ITKElastix)

In [1]:
# Update sys.path to reference src/ modules
import os
import sys
import copy
import importlib
from urllib.request import urlretrieve

import itk
from itkwidgets import view, checkerboard, compare
from ipywidgets import FloatProgress, Label, HBox, VBox, FloatText, ColorPicker, Button
PATTERN_COUNT = 5

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
os.makedirs('Input', exist_ok=True)
os.makedirs('Output', exist_ok=True)

In [3]:
meshes = ['901','902','906','907','908','915','916','917','918']
MESH_TO_USE = meshes[0]

TARGET_MESH_FILE = f'Input/{MESH_TO_USE}-R-mesh.vtk'
# Future update will replace with smaller template
TEMPLATE_MESH_FILE = f'Input/901-L-mesh.vtk'

MEANSQUARES_OUTPUT_FILE = f'Output/{MESH_TO_USE}-meansquares-registered.obj'
DIFFEO_OUTPUT_FILE = f'Output/{MESH_TO_USE}-diffeo-registered.obj'
ELASTIX_OUTPUT_FILE = f'Output/{MESH_TO_USE}-elastix-registered.obj'
POINTSET_OUTPUT_FILE = f'Output/{MESH_TO_USE}-pointset-registered.obj'

In [4]:
# Download meshes
if not os.path.exists(TARGET_MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/5f9daaba50a41e3d1924dae9/download'
    urlretrieve(url, TARGET_MESH_FILE)
if not os.path.exists(TEMPLATE_MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/5f9daaae50a41e3d1924dae1/download'
    urlretrieve(url, TEMPLATE_MESH_FILE)

In [5]:
template_mesh = itk.meshread(TEMPLATE_MESH_FILE, itk.F)
target_mesh = itk.meshread(TARGET_MESH_FILE, itk.F)

### Compare images with ITKWidgets

We can use `view`, `compare`, and `checkerboard` to inspect mesh and image data.

In [ ]:
view(geometries=[template_mesh,target_mesh])

## Mesh-To-Image Conversion in the Base Class
Python registration classes inheriting from `MeshToMeshRegistrar` implement unique registration algorithms. The base class includes common definitions for 3D mesh-to-mesh registration, abstract methods, and a mesh-to-image conversion method.

The `mesh_to_image` function takes in a 3D mesh and converts it into an ITK 3D image object. The spacing, origin, and size of the 3D image may be calculated from the minimum bounding box of the mesh or set from a reference image.

In [6]:
from src.hasi.hasi.meshtomeshregistrar import MeshToMeshRegistrar
registrar = MeshToMeshRegistrar()

In [7]:
template_image = registrar.mesh_to_image(template_mesh)

In [8]:
target_image = registrar.mesh_to_image(target_mesh, template_image)

Comparing the meshes generated from the given meshes, we see that the two bone images are generally similar but do not exactly line up together. Registration will translate one mesh so that the two bones better coincide.

In [9]:
# TODO second image does not match first image
checkerboard(template_image, target_image, pattern=PATTERN_COUNT)

## Point set downsizing in the Base Class

The base `MeshToMeshRegistrar` class also provides functionality for uniform random point set sampling from a given mesh, primarily used to improve performance for point set based registration of dense meshes. Note that uniform random sampling may not be suitable for all shapes, in which case application-specific resampling should be carried out externally prior to registration.

In [10]:
target_points_reduced = \
    registrar.randomly_sample_mesh_points(mesh=target_mesh, sampling_rate=0.01)

In [ ]:
view(geometries=[target_mesh],point_sets=[target_points_reduced])

## Run Mean Squares Image Registration

The `MeanSquaresRegistrar` class converts meshes to images and runs [Broyden-Fletcher-Goldfarb-Shanno Optimization](https://itk.org/Doxygen/html/classitk_1_1LBFGSBOptimizerv4.html) on a [BSplineTransform](https://itk.org/Doxygen/html/classitk_1_1BSplineTransform.html) to iteratively reduce the mean square error. The resulting transform is then applied to resample the target mesh into the template mesh domain.

Progress is shown with an itkwidgets display via hooks into the ITK event-observer system. The resultant mesh is returned as an object in the Python environment and may be optionally written out to a file. Iteration updates may also be printed to the output window with the optional `verbose` flag.

In [11]:
from src.hasi.hasi.meansquaresregistrar import MeanSquaresRegistrar

In [12]:
# Must instantiate a registration object to initialize optimizers
registrar = MeanSquaresRegistrar()

In [13]:
progress = FloatProgress(
        min=0.0,
        max=150.0,
        step=1
    )
box = HBox([
    Label('Register images'),
    progress
])
box

In [14]:
def update_progress():
    progress.value = registrar.optimizer.GetCurrentIteration()
registrar.optimizer.AddObserver(itk.IterationEvent(), update_progress)

0

In [15]:
(transform, mesh_result) = registrar.register(template_mesh,
                                             target_mesh,
                                             filepath=MEANSQUARES_OUTPUT_FILE,
                                             verbose=True,
                                             num_iterations=200)

0 0.03902562455535499 0.0
0 0.02843709659649114 0.0
0 0.02843709659649114 0.0
1 0.021534717551505798 0.0009582783977029718
1 0.021534717551505798 0.0009582783977029718
2 0.01790224911093285 0.00043193967453070587
2 0.01790224911093285 0.00043193967453070587
3 0.01568735406387658 0.00024653588676924737
3 0.01568735406387658 0.00024653588676924737
4 0.012572812905498177 0.0001938980902412675
4 0.012572812905498177 0.0001938980902412675
5 0.010982249945285367 0.00015319597818792382
5 0.010982249945285367 0.00015319597818792382
6 0.010363658471730493 0.0001203197280705292
6 0.010363658471730493 0.0001203197280705292
7 0.00917110184613044 0.00012346212100002512
7 0.00917110184613044 0.00012346212100002512
8 0.008587382362441122 0.00010581326627687776
8 0.008587382362441122 0.00010581326627687776
9 0.007903788433334932 0.00023582429403319085
9 0.007903788433334932 0.00023582429403319085
10 0.007710042007798568 7.71329623659586e-05
10 0.007710042007798568 7.71329623659586e-05
11 0.00748263845

Comparison of the resulting mesh with the target shows successful registration.

In [ ]:
view(geometries=[meansquares_mesh_result,moving_mesh])

Comparison of the resulting mesh with the output file shows congruency.

In [16]:
file_mesh_result = itk.meshread(MEANSQUARES_OUTPUT_FILE, itk.F)

In [ ]:
view(geometries=[file_mesh_result,meansquares_mesh_result])

## Diffeomorphic Registration

The `DiffeoRegistrar` class converts meshes to images and performs registration using the [Diffeomorphic Demons registration algorithm](https://itk.org/Doxygen/html/classitk_1_1DiffeomorphicDemonsRegistrationFilter.html). The resultant deformation field is then applied to resample the target mesh into the template mesh domain.

Custom observers may print out iteration data accessed via the `registrar.filter` object.

In [17]:
from src.hasi.hasi.diffeoregistrar import DiffeoRegistrar

In [18]:
registrar = DiffeoRegistrar()

In [19]:
diffeoProgress = FloatProgress(
        min=0.0,
        max=200.0,
        step=1
    )
diffeoBox = HBox([
    Label('Register images'),
    diffeoProgress
])
diffeoBox

In [20]:
def update_diff_progress():
    diffeoProgress.value = registrar.filter.GetElapsedIterations()

registrar.filter.AddObserver(itk.IterationEvent(),update_diff_progress)

(transform, mesh_result) = registrar.register(template_mesh,
                                        target_mesh,
                                        filepath=DIFFEO_OUTPUT_FILE,
                                        verbose=True)

1.7976931348623157e+308
0.03902562456210709
0.03620058111051529
0.03385740047695736
0.0316529992799868
0.029671274883454234
0.027863336769245968
0.02626372137143445
0.024784571211718686
0.02343911289037383
0.022179969985074023
0.021160217575979016
0.02022989923756486
0.019367058349974004
0.018567845127124925
0.017848969406583762
0.01718766149634235
0.01657368277679982
0.01599817637581068
0.01545783158564288
0.014948025199071284
0.014461622847641745
0.014000352105777526
0.013558082766414237
0.01311751745327867
0.012612455200814074
0.012162631078637215
0.011740243666151801
0.011309865240838797
0.01089920347696751
0.010521650813570736
0.010196561107985154
0.009891130130343637
0.009600816910980126
0.009315760503390665
0.009039516487389243
0.008767196010894804
0.008506293948833205
0.008248633969299386
0.00799543225258579
0.0077453645420522965
0.007497301744479739
0.007257772289115741
0.007027261536318484
0.0068053907333040725
0.006592009351901684
0.006385407763355721
0.006188817973227064
0.

Compare the translated image to the target image.

In [ ]:
#diffeo_image_result = registrar.mesh_to_image(diffeo_mesh_result)
#checkerboard(diffeo_image_result,moving_img,pattern=PATTERN_COUNT)
view(geometries=[mesh_result, target_mesh])

Comparison of the resulting mesh with the output file shows congruency.

In [ ]:
file_mesh_result = itk.meshread(DIFFEO_OUTPUT_FILE, itk.F)
#file_image_result = registrar.mesh_to_image(file_mesh_result)
#checkerboard(diffeo_image_result,file_image_result,pattern=PATTERN_COUNT)
view(geometries=[file_mesh_result,mesh_result])

# Entropy-based Registration

The `PointSetEntropyRegistrar` class registers two 3D meshes by converting them to point clouds and computing the transform which minimizes entropy measures between the two clouds. In this example we substitute a [`EuclideanDistancePointSetToPointSetMetric`](https://itk.org/Doxygen/html/classitk_1_1EuclideanDistancePointSetToPointSetMetricv4.html) to compare the two clouds.

In [21]:
from src.hasi.hasi.pointsetentropyregistrar import PointSetEntropyRegistrar
registrar = PointSetEntropyRegistrar()

In [22]:
progress = FloatProgress(
        min=0.0,
        max=2000.0,
        step=1
    )
progressBox = HBox([
    Label('Register images'),
    progress
])
progressBox

In [23]:
def update_progress():
    progressBox.value = registrar.optimizer.GetCurrentIteration()

registrar.optimizer.AddObserver(itk.IterationEvent(),update_progress)

0

In [25]:
metric = itk.EuclideanDistancePointSetToPointSetMetricv4[itk.PointSet[itk.F,3]].New()

(transform, mesh_result) = registrar.register(template_mesh,
                       target_mesh,
                       metric=metric,
                       filepath=POINTSET_OUTPUT_FILE,
                       resample_rate=0.005,
                       verbose=True,
                       learning_rate=1.0,
                       max_iterations=200,
                       resample_from_target=False)

0 0.11653694013677242
0 0.11653694013677242
0 0.11653694013677242
0 0.11653694013677242
0 0.11653694013677242
0 0.14299087349649367
0 0.14299087349649367
1 0.1348841081511713
1 0.1348841081511713
2 0.12898271315311685
2 0.12898271315311685
3 0.1246315007837364
3 0.1246315007837364
4 0.1212819745835345
4 0.1212819745835345
5 0.1188246510044722
5 0.1188246510044722
6 0.1167187295839064
6 0.1167187295839064
7 0.11487317279098527
7 0.11487317279098527
8 0.11329302557318019
8 0.11329302557318019
9 0.11189197105622001
9 0.11189197105622001
10 0.11063591302518083
10 0.11063591302518083
11 0.10948059642565089
11 0.10948059642565089
12 0.10847938077972978
12 0.10847938077972978
13 0.10758298309564214
13 0.10758298309564214
14 0.10676379456645364
14 0.10676379456645364
15 0.10598345913678081
15 0.10598345913678081
16 0.10526402003158299
16 0.10526402003158299
17 0.10457306585168157
17 0.10457306585168157
18 0.10387656202239602
18 0.10387656202239602
19 0.10321961004986056
19 0.10321961004986056


Wrote resulting mesh to Output/901-pointset-registered.obj


In [26]:
template_resampled = \
    registrar.resample_template_from_target(mesh_result, target_mesh)

In [27]:
itk.meshwrite(template_resampled,'Output/pointset-resampled.obj')

In [ ]:
view(geometries=[target_mesh,mesh_result,template_resampled])

In [ ]:
# Clean up file output
os.remove(MEANSQUARES_OUTPUT_FILE)
os.remove(DIFFEO_OUTPUT_FILE)
os.remove(ELASTIX_OUTPUT_FILE)
os.remove(POINTSET_OUTPUT_FILE)